<a href="https://colab.research.google.com/github/beeanerd/water-clarity-detection/blob/main/water_clarity_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python numpy imutils argparse


In [6]:
!cd drive/MyDrive/images/ 

In [8]:
import argparse
import math
import cv2 as cv
import numpy as np
from imutils import paths

In [9]:
def blur_scoring(image):
    return cv.Laplacian(image, cv.CV_64F).var()

In [10]:
def save_image(image_list):                     # image_list = (name/path, img file)
    good_saves = 0
    bad_saves = 0
    for image_tuples in image_list:
        for name, image in image_tuples:
            check = cv.imwrite(name, image)
            if check:
                good_saves += 1
            else:
                print(f"{name} failed to save")
                bad_saves += 1
    print(f"Proper saves: {good_saves}\nFailed saved: {bad_saves}")

In [11]:
def hough_line_drawing(images):
    image_files_to_save = list()
    for count, imgPath in enumerate(images):
        image = cv.imread(imgPath)
        image2 = np.copy(image)
        print(imgPath)
        # gray = cv.imread(cv.samples.findFile(imgPath), cv.IMREAD_GRAYSCALE)
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        print("gray")
        blur = cv.medianBlur(gray, 5)
        print("blur")
        adapt_type = cv.ADAPTIVE_THRESH_GAUSSIAN_C
        thresh_type = cv.THRESH_BINARY_INV
        threshold = cv.adaptiveThreshold(blur, 255, adapt_type, thresh_type, 41, 20)
        # threshold, image_result = cv.threshold(blur, 255, 0, cv.THRESH_BINARY + cv.THRESH_OTSU)
        sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        fm = cv.filter2D(threshold, -1, sharpen_kernel)
        print("adaptiveThreshold")
        # fm = variance_of_laplacian(gray)                                        ### Used to calculate the blurriness
        # fm = cv.Canny(image, 50, 200)                                           ### Canny looks promising but it restricts too much, maybe play with thresh?
        rho, theta, thresh = 2, np.pi / 180, 400
        lines = cv.HoughLines(fm, rho, theta, thresh)
        if lines is not None:
            for i in range(0, len(lines)):
                rho = lines[i][0][0]
                theta = lines[i][0][1]
                a = math.cos(theta)
                b = math.sin(theta)
                x0 = a * rho
                y0 = b * rho
                pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
                pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
                cv.line(image2, pt1, pt2, (0, 0, 255), 3, cv.LINE_AA)
        rho, theta, thresh = 2, np.pi / 180, 400
        lines_p = cv.HoughLinesP(fm, 1, np.pi / 180, 50, None, 50, 10)
        if lines_p is not None:
            for i in range(0, len(lines_p)):
                l = lines_p[i][0]
                cv.line(image, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv.LINE_AA)
        name = (f"Probability Hough Line - Standard Preprocessing - {imgPath}", image)
        name2 = (f"Hough Line - Standard Preprocessing - {imgPath}", image2)
        name3 = (f"B&W Image - Standard Preprocessing - {imgPath}", fm)
        image_files_to_save.append((name, name2, name3))
        cv.imshow("Probability Hough Line Standard Preprocessing", image)
        cv.putText(fm, f"{fm}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
        cv.imshow("Hough Line with Standard Preprocessing", image2)
        cv.imshow("B&W Image", fm)
        key = cv.waitKey(0)
    return image_files_to_save

In [12]:
def main():
    # This is going to process the image and set it to output purely the edges, plan is to run hough line detection on this
    # Which will maybe fix the issue with to many lines being represented

    ap = argparse.ArgumentParser()
    ap.add_argument("-i", "--images", required=True, help="path to input directory of images")
    args = vars(ap.parse_args())
    images = paths.list_images(args["images"])
    # thresh_vals = otsu_binarization(images)
    to_save = hough_line_drawing(images)
    save_image(to_save)

In [13]:
if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] -i IMAGES
ipykernel_launcher.py: error: the following arguments are required: -i/--images


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
!python main.py --images "/drive/MyDrive/images/"

python3: can't open file 'main.py': [Errno 2] No such file or directory
